# 네이버 블로그 크롤링 - 

In [29]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [30]:
chromedriver = '/Users/sku11/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)


In [31]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('스마트폰')
driver.get(url)

# 페이지를 아래로 내려서 더 많은 정보 확보

In [32]:
#blog
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [34]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [43]:
lis = soup.select('li.bx._svp_item')
len(lis)


62

In [42]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, content, date

('[스마트폰] 삼성 갤럭시Z 플립3 언박싱 후기',
 '진짜 역대급으로 나온 스마트폰, 삼성 갤럭시Z 플립3 언박싱 후기 함께 공개된 삼성 갤럭시Z 폴드3가 기대... 아무래도 기존 스마트폰과 거의 비슷한 비율과 함께 큰 화면을 반으로 접어서 가지고 다니면서 컴팩트하게...',
 '2021.08.16.')

In [44]:
title_list, content_list, date_list = [], [], []
for li in lis:
    #광고 삭제
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip() == '광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()
        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

In [45]:
df = pd.DataFrame({
   'title' : title_list, 'content' : content_list, 'date':date_list

})

In [46]:
df.head()

,title,content,date
0,[스마트폰] 삼성 갤럭시Z 플립3 언박싱 후기,"진짜 역대급으로 나온 스마트폰, 삼성 갤럭시Z 플립3 언박싱 후기 함께 공개된 삼성...",2021.08.16.
1,갤럭시 A03s의 출시소식과 상세 정보: 개발도상국용 초저가 스마트폰,삼성이 인도시장 등 개발도상국용 초저가 스마트폰인 A0x 시리즈를 지난 2019년부...,7일 전
2,에어팟 프로 안드로이드 스마트폰과 연결하는 방법,"그러면 이때 스마트폰 블루투스 메뉴로 가줍니다. 찾기를 눌러주면, 이렇게 검색이 되...",2021.08.21.
3,갤럭시 굿락 루틴+ 스마트폰 배터리 충전 구간 알림,갤럭시 스마트폰을 사용하고 있다면 굿락 앱은 꼭 써볼 것을 추천드려요. 지속적인 업...,2021.08.08.
4,삼성 갤럭시노트10 가격 2021년에도 매력적인 스마트폰,"일상에서 스마트폰 사용량은 점점 늘어나고 있습니다. 카카오톡, 라인 같은 기본적인 ...",2021.06.23.


In [48]:
df.to_csv('smt_blog.txt')